In [11]:
import pandas as pd
import seaborn as sns

In [62]:
# read in data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
file_path = 'https://raw.githubusercontent.com/Lambda-School-Labs/Labs25-Bridges_to_Prosperity-TeamC-ds/main/B2P%20Rwanda%20Site%20Assessment%20Data_2020.06.03.csv'
df = pd.read_csv(file_path, encoding='latin-1')
print(f'Dimensions of dataset: {df.shape}')
df.head()

Dimensions of dataset: (1472, 27)


,Country,Province,District,Sector,Cell,Bridge Site Name,Project Stage,Project Sub-Stage,Project Code,Bridge Type,Span (m),GPS (Latitude),GPS (Longitude),Individuals Directly Served,Form: Form Name,Community Served 1,Community Served 2,Community Served 3,Community Served 4,Community Served 5,Community Served 6,Community Served 7,Community Served 8,Community Served 9,Community Served 10,CaseSafeID Form,Bridge Opportunity: Opportunity ID
0,Rwanda,Western Province,Rusizi,Giheke,Gakomeye,Buzi,Rejected,Technical,1014107,Suspended,NaN,-2.420560,28.966200,NaN,Project Assessment - 2018.10.29,Buzi,Kabuga,Kagarama,Gacyamo,Gasheke,NaN,NaN,NaN,NaN,NaN,a1if1000002e51bAAA,006f100000d1fk1
1,Rwanda,Western Province,Rusizi,Giheke,Gakomeye,Kamigisha,Rejected,Technical,1014106,Suspended,NaN,-2.424860,28.957260,NaN,Project Assessment - 2018.10.29,Kabuga,Buzi,Gacyamo - Gasheke,Kagarama,Kanoga,NaN,NaN,NaN,NaN,NaN,a1if1000002e51WAAQ,006f100000d1fjw
2,Rwanda,Northern Province,Rulindo,Buyoga,Gahororo-Gipfundo,Gipfundo,Rejected,Technical,1007651,Suspended,8.0,-1.720530,30.081240,NaN,Project Assessment - 2018.8.11,Gapfundo,Rusabira ( Gicumbi),Merezo,Minanire,Karutongo,NaN,NaN,NaN,NaN,NaN,a1if10000025nz8AAA,006f100000a86I3
3,Rwanda,Northern Province,Gicumbi,Kageyo,Kabuga/gatobotobo,Nyarubande,Rejected,NaN,1012493,Other,NaN,-1.655950,30.078840,NaN,Project Assessment - 2018.8.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a1if10000025nzDAAQ,006f100000cPpL8
4,Rwanda,Southern Province,Kamonyi,Kayenzi,Kirwa,Gisizi,Identified,Requested,1014318,Suspended,NaN,-1.870868,29.877686,NaN,Project Assessment - 2018.11.15,Gisizi,Nyabitare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a1if1000002gMwRAAU,006f100000eescb


In [63]:
# split the Form: Form Name to get the data from the column and convert date to datetime

df['Assessment Date'] = df['Form: Form Name'].str.replace('Project Assessment - ', "")

df['Assessment Date'] = pd.to_datetime(df['Assessment Date'])
print(f'Dimensions of dataset: {df.shape}')
df.head()

Dimensions of dataset: (1472, 28)


,Country,Province,District,Sector,Cell,Bridge Site Name,Project Stage,Project Sub-Stage,Project Code,Bridge Type,Span (m),GPS (Latitude),GPS (Longitude),Individuals Directly Served,Form: Form Name,Community Served 1,Community Served 2,Community Served 3,Community Served 4,Community Served 5,Community Served 6,Community Served 7,Community Served 8,Community Served 9,Community Served 10,CaseSafeID Form,Bridge Opportunity: Opportunity ID,Assessment Date
0,Rwanda,Western Province,Rusizi,Giheke,Gakomeye,Buzi,Rejected,Technical,1014107,Suspended,NaN,-2.420560,28.966200,NaN,Project Assessment - 2018.10.29,Buzi,Kabuga,Kagarama,Gacyamo,Gasheke,NaN,NaN,NaN,NaN,NaN,a1if1000002e51bAAA,006f100000d1fk1,2018-10-29
1,Rwanda,Western Province,Rusizi,Giheke,Gakomeye,Kamigisha,Rejected,Technical,1014106,Suspended,NaN,-2.424860,28.957260,NaN,Project Assessment - 2018.10.29,Kabuga,Buzi,Gacyamo - Gasheke,Kagarama,Kanoga,NaN,NaN,NaN,NaN,NaN,a1if1000002e51WAAQ,006f100000d1fjw,2018-10-29
2,Rwanda,Northern Province,Rulindo,Buyoga,Gahororo-Gipfundo,Gipfundo,Rejected,Technical,1007651,Suspended,8.0,-1.720530,30.081240,NaN,Project Assessment - 2018.8.11,Gapfundo,Rusabira ( Gicumbi),Merezo,Minanire,Karutongo,NaN,NaN,NaN,NaN,NaN,a1if10000025nz8AAA,006f100000a86I3,2018-08-11
3,Rwanda,Northern Province,Gicumbi,Kageyo,Kabuga/gatobotobo,Nyarubande,Rejected,NaN,1012493,Other,NaN,-1.655950,30.078840,NaN,Project Assessment - 2018.8.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a1if10000025nzDAAQ,006f100000cPpL8,2018-08-11
4,Rwanda,Southern Province,Kamonyi,Kayenzi,Kirwa,Gisizi,Identified,Requested,1014318,Suspended,NaN,-1.870868,29.877686,NaN,Project Assessment - 2018.11.15,Gisizi,Nyabitare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a1if1000002gMwRAAU,006f100000eescb,2018-11-15


In [64]:
df.dtypes

Country                                       object
Province                                      object
District                                      object
Sector                                        object
Cell                                          object
Bridge Site Name                              object
Project Stage                                 object
Project Sub-Stage                             object
Project Code                                  object
Bridge Type                                   object
 Span (m)                                    float64
 GPS (Latitude)                              float64
GPS (Longitude)                              float64
Individuals Directly Served                  float64
Form: Form Name                               object
Community Served 1                            object
Community Served 2                            object
Community Served 3                            object
Community Served 4                            

In [55]:
# what are the duplicate project codes?
# it looks like that all the duplicates have two values. One has three.
# a quick and simple way to remove the duplicates would keep the sorting by project code and assesment date
# then drop the duplicates and keep the first occurence. That way we have the most recent date. 

# this was our test to find all the duplicates that we needed to drop
# testing = df[df.duplicated(subset='Project Code', keep=False)].sort_values(by=['Project Code', 'Assessment Date'], ascending=False)
# testing.head()
# testing[testing['Individuals Directly Served'].notna()]

# Indices that we 

In [65]:
# manually went thru above duplicates and selected those that needed to be DROPPED. 

indexes_to_drop = [192,210,216,767,202,1119,908,735,901,272,643,756,
                   738,193,655,642,327,909,1274,1289,135,176,1213,
                   493,178,205,387,397,395,398,399,401,372,371,892,
                   839,776,171,169,170,1117,1096,771,1109,95,89,132,133,134,
                   ]


df = df.drop(index=indexes_to_drop)
print(f'Dimensions of dataset: {df.shape}')

Dimensions of dataset: (1423, 28)


In [67]:
# make sure there are no duplicates 

df.duplicated(subset='Project Code', keep=False).value_counts()

False    1423
dtype: int64

In [6]:
# there are some columns that are all null values, we can drop those.
df.isnull().sum()


Country                                  0
Province                                 0
District                                 0
Sector                                   3
Cell                                    51
Bridge Site Name                         0
Project Stage                            0
Project Sub-Stage                       72
Project Code                             0
Bridge Type                            177
 Span (m)                             1064
 GPS (Latitude)                          4
GPS (Longitude)                          4
Individuals Directly Served           1421
Form: Form Name                          0
Community Served 1                      97
Community Served 2                     208
Community Served 3                     598
Community Served 4                     828
Community Served 5                    1070
Community Served 6                    1472
Community Served 7                    1472
Community Served 8                    1472
Community S

In [7]:
# these are columns we can drop
col_drop = ['Community Served 6', 'Community Served 7',
            'Community Served 8', 'Community Served 9',
            'Community Served 10'
           ]

# these are columns that we want as Ids when we melt the table
ID_variable = ['Country','Province', 'District', 'Sector', 'Cell', 
               'Bridge Site Name', 'Project Stage', 
               'Project Sub-Stage', 'Project Code', 'Bridge Type',
               ' Span (m)', ' GPS (Latitude)', 'GPS (Longitude)',
               'Individuals Directly Served', 'Form: Form Name',
               'CaseSafeID Form', 'Bridge Opportunity: Opportunity ID',
               'Assessment Date'
              ]

# these are the columns that we want to melt to values
value_variables = ['Community Served 1', 'Community Served 2',
                   'Community Served 3', 'Community Served 4',
                   'Community Served 5'
                  ]

In [8]:
# melt the dataframe so we get all the communities that a bridge would serve in one column
# this will make duplicate observations of bridges. THis is okay since we want to be able to get counts
# of how many villages a bridge would service.

melted_df = pd.melt(df, id_vars=ID_variable, value_vars=value_variables, var_name='Original_Community_col', value_name='Community_Served', ignore_index=False )
print(f'Dimensions of dataset: {melted_df.shape}')
melted_df.head()

Dimensions of dataset: (7360, 20)


,Country,Province,District,Sector,Cell,Bridge Site Name,Project Stage,Project Sub-Stage,Project Code,Bridge Type,Span (m),GPS (Latitude),GPS (Longitude),Individuals Directly Served,Form: Form Name,CaseSafeID Form,Bridge Opportunity: Opportunity ID,Assessment Date,Original_Community_col,Community_Served
0,Rwanda,Western Province,Rusizi,Giheke,Gakomeye,Buzi,Rejected,Technical,1014107,Suspended,NaN,-2.420560,28.966200,NaN,Project Assessment - 2018.10.29,a1if1000002e51bAAA,006f100000d1fk1,2018-10-29,Community Served 1,Buzi
1,Rwanda,Western Province,Rusizi,Giheke,Gakomeye,Kamigisha,Rejected,Technical,1014106,Suspended,NaN,-2.424860,28.957260,NaN,Project Assessment - 2018.10.29,a1if1000002e51WAAQ,006f100000d1fjw,2018-10-29,Community Served 1,Kabuga
2,Rwanda,Northern Province,Rulindo,Buyoga,Gahororo-Gipfundo,Gipfundo,Rejected,Technical,1007651,Suspended,8.0,-1.720530,30.081240,NaN,Project Assessment - 2018.8.11,a1if10000025nz8AAA,006f100000a86I3,2018-08-11,Community Served 1,Gapfundo
3,Rwanda,Northern Province,Gicumbi,Kageyo,Kabuga/gatobotobo,Nyarubande,Rejected,NaN,1012493,Other,NaN,-1.655950,30.078840,NaN,Project Assessment - 2018.8.11,a1if10000025nzDAAQ,006f100000cPpL8,2018-08-11,Community Served 1,NaN
4,Rwanda,Southern Province,Kamonyi,Kayenzi,Kirwa,Gisizi,Identified,Requested,1014318,Suspended,NaN,-1.870868,29.877686,NaN,Project Assessment - 2018.11.15,a1if1000002gMwRAAU,006f100000eescb,2018-11-15,Community Served 1,Gisizi


In [9]:
# now that we have our melted dataframe, we want to delete any observation that has a 
# null value in the 'Community_Served' column

melted_df = melted_df[melted_df['Community_Served'].notna()]
print(f'Dimensions of dataset: {melted_df.shape}')
melted_df.head()

Dimensions of dataset: (4559, 20)


,Country,Province,District,Sector,Cell,Bridge Site Name,Project Stage,Project Sub-Stage,Project Code,Bridge Type,Span (m),GPS (Latitude),GPS (Longitude),Individuals Directly Served,Form: Form Name,CaseSafeID Form,Bridge Opportunity: Opportunity ID,Assessment Date,Original_Community_col,Community_Served
0,Rwanda,Western Province,Rusizi,Giheke,Gakomeye,Buzi,Rejected,Technical,1014107,Suspended,NaN,-2.420560,28.966200,NaN,Project Assessment - 2018.10.29,a1if1000002e51bAAA,006f100000d1fk1,2018-10-29,Community Served 1,Buzi
1,Rwanda,Western Province,Rusizi,Giheke,Gakomeye,Kamigisha,Rejected,Technical,1014106,Suspended,NaN,-2.424860,28.957260,NaN,Project Assessment - 2018.10.29,a1if1000002e51WAAQ,006f100000d1fjw,2018-10-29,Community Served 1,Kabuga
2,Rwanda,Northern Province,Rulindo,Buyoga,Gahororo-Gipfundo,Gipfundo,Rejected,Technical,1007651,Suspended,8.0,-1.720530,30.081240,NaN,Project Assessment - 2018.8.11,a1if10000025nz8AAA,006f100000a86I3,2018-08-11,Community Served 1,Gapfundo
4,Rwanda,Southern Province,Kamonyi,Kayenzi,Kirwa,Gisizi,Identified,Requested,1014318,Suspended,NaN,-1.870868,29.877686,NaN,Project Assessment - 2018.11.15,a1if1000002gMwRAAU,006f100000eescb,2018-11-15,Community Served 1,Gisizi
5,Rwanda,Southern Province,Kamonyi,Kayenzi,Kirwa,Ruheka,Rejected,Technical,1014319,Suspended,NaN,-1.883957,29.850548,NaN,Project Assessment - 2018.11.15,a1if1000002gMwqAAE,006f100000eescl,2018-11-15,Community Served 1,Ruheka


In [10]:
# lets check to see if there are still null values that we care about in the dataset

melted_df.isnull().sum()


Country                                  0
Province                                 0
District                                 0
Sector                                   3
Cell                                    59
Bridge Site Name                         0
Project Stage                            0
Project Sub-Stage                      212
Project Code                             0
Bridge Type                            202
 Span (m)                             3438
 GPS (Latitude)                          2
GPS (Longitude)                          2
Individuals Directly Served           4458
Form: Form Name                          0
CaseSafeID Form                          0
Bridge Opportunity: Opportunity ID       0
Assessment Date                          0
Original_Community_col                   0
Community_Served                         0
dtype: int64